In [33]:
import pandas as pd

In [34]:
df = pd.read_csv('airline data/april_2019/On_Time_Reporting_Carrier_On_Time_Performance_(1987_present)_2019_4.csv')

c:\Users\reach\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (48,76,77,84) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [35]:
df_div = df.filter(like="Div")
removable = list(df_div.columns)
df = df.drop(removable, axis=1)
df = df.dropna(axis=1, how='all')

In [36]:
def print_columns(df: pd.DataFrame):
    for column in df.columns:
        print(column)

In [37]:
def get_data_by_airport(df: pd.DataFrame, city: str) -> pd.DataFrame:
    return df[(df['Origin'].str.contains(city) == True)]


In [38]:
cities = ['JFK', 'DFW', 'MCO', 'LAX', 'SEA']
cleaned_df = pd.DataFrame()
for city in cities:
    cleaned_df = cleaned_df.append(get_data_by_airport(df, city))

df = cleaned_df

In [39]:
removables = [
    'Year','Tail_Number', 'OriginAirportID',
    'OriginAirportSeqID','OriginCityMarketID','OriginCityName',
    'OriginState','OriginStateFips','OriginStateName','OriginWac', 
    'DestAirportID','DestAirportSeqID','DestCityMarketID',
    'DestCityName','DestState','DestStateFips','DestStateName',
    'DestWac','DepTime', 'DepDelayMinutes', 'DepDel15',
    'DepartureDelayGroups','DepTimeBlk','WheelsOff','WheelsOn',
    'CRSArrTime','ArrTime','ArrDelay','ArrDelayMinutes','ArrDel15',
    'ArrivalDelayGroups','ArrTimeBlk','Cancelled','CancellationCode',
    'Flights','CarrierDelay','WeatherDelay','NASDelay',
    'SecurityDelay','LateAircraftDelay','FirstDepTime',
    'TotalAddGTime','LongestAddGTime'
]
for remove in removables:
    try:
        df = df.drop(remove, axis=1)
    except Exception as e:
        print(f'Already Removed {remove} or {e}')

In [40]:
def one_hot_encode(df: pd.DataFrame, column_name: str) -> pd.DataFrame:
    try:
        dummy1 = pd.get_dummies(df[column_name])
        dummy1 = dummy1.add_prefix(f"{column_name} - ")
        df = pd.concat([df, dummy1], axis=1).drop(column_name, axis=1)
        df.head()
    except Exception as e:
        print(f"Error in one_hot_encode: {e}")
    return df

In [41]:
one_hot_columns = [
    'Reporting_Airline', 'IATA_CODE_Reporting_Airline', 'DOT_ID_Reporting_Airline', 'Dest'
    ]
for col in one_hot_columns:
    df = one_hot_encode(df, col)

In [43]:
df['DepDelay'] = [0 if temp <= 0 else 1 for temp in df['DepDelay']]

In [44]:
df['label'] = df['DepDelay']
df = df.drop('DepDelay', axis=1)

In [46]:
weather_df = pd.read_csv('weather_data.csv')

In [47]:
names = {
    'LOS ANGELES INTERNATIONAL AIRPORT, CA US': 'LAX',
    'SEATTLE TACOMA AIRPORT, WA US': 'SEA',
    'ORLANDO INTERNATIONAL AIRPORT, FL US': 'MCO',
    'BUENA VENTURA LAKES 6.0 ENE, FL US': 'MCO',
    'DAL FTW WSCMO AIRPORT, TX US': 'DFW',
    'JFK INTERNATIONAL AIRPORT, NY US': 'JFK',
}
weather_df['NAME'] = [names[var] for var in weather_df['NAME']]

In [48]:
weather_removables = [
    'STATION','AWND_ATTRIBUTES','DAPR','DAPR_ATTRIBUTES','MDPR',
    'MDPR_ATTRIBUTES','PGTM', 'PGTM_ATTRIBUTES','TAVG_ATTRIBUTES',
    'TMAX_ATTRIBUTES','TMIN_ATTRIBUTES','WDF2_ATTRIBUTES','WDF5_ATTRIBUTES',
    'WSF2_ATTRIBUTES','WSF5_ATTRIBUTES','WT01','WT02','WT03','WT04','WT05',
    'WT05_ATTRIBUTES','WT06','WT06_ATTRIBUTES','WT07','WT07_ATTRIBUTES',
    'WT08','WT09','WT09_ATTRIBUTES' 
]
for remove in weather_removables:
    try:
        weather_df = weather_df.drop(remove, axis=1)
    except Exception as e:
        print(f'Already Removed {remove} or {e}')

In [49]:
weather_one_hot_columns = [
    'PRCP_ATTRIBUTES','SNOW_ATTRIBUTES','SNWD_ATTRIBUTES','WT01_ATTRIBUTES',
    'WT02_ATTRIBUTES','WT03_ATTRIBUTES','WT04_ATTRIBUTES','WT08_ATTRIBUTES'
]
for col in weather_one_hot_columns:
    weather_df = one_hot_encode(weather_df, col)

In [152]:
weather_df['DATE'] = pd.to_datetime(weather_df['DATE'])
df['DATE'] = pd.to_datetime(df['DATE'])

In [59]:
df = df.rename(columns={'Origin':'NAME', 'FlightDate':'DATE'})

In [160]:
def join_by_date(df: pd.DataFrame, weather_df: pd.DataFrame, airport_name: str):
    tmp = df.loc[df['NAME'] == airport_name]
    tmp = tmp.drop(['NAME'], axis=1)
    weather_tmp = weather_df.loc[weather_df['NAME'] == airport_name]
    return pd.merge(tmp, weather_tmp, how='outer', on='DATE')

In [165]:
cleaned_df = pd.DataFrame()
for city in cities:
    cleaned_df = cleaned_df.append(join_by_date(df, weather_df, city))

cleaned_df = cleaned_df.drop(['DATE'], axis=1)
cleaned_df = one_hot_encode(cleaned_df, 'NAME')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75351 entries, 0 to 11220
Columns: 316 entries, Quarter to NAME - SEA
dtypes: float64(294), uint8(22)
memory usage: 171.2 MB


In [166]:
cleaned_df.to_csv('cleaned_airline_data/clean_merge_april.csv')

In [278]:
df = df.reset_index(drop=True)
df = df.dropna(how='any')
df.to_csv('cleaned_airline_data/clean_april.csv')